# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-16 12:50:45] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=30549, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=172591412, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1

[2025-06-16 12:50:59] Attention backend not set. Use fa3 backend by default.
[2025-06-16 12:50:59] Init torch distributed begin.


[2025-06-16 12:51:00] Init torch distributed ends. mem usage=0.00 GB


[2025-06-16 12:51:00] Load weight begin. avail mem=47.95 GB


[2025-06-16 12:51:01] Using model weights format ['*.safetensors']
[2025-06-16 12:51:01] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-06-16 12:51:02] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=45.11 GB


[2025-06-16 12:51:02] INFO:     Started server process [3788829]
[2025-06-16 12:51:02] INFO:     Waiting for application startup.
[2025-06-16 12:51:02] INFO:     Application startup complete.
[2025-06-16 12:51:02] INFO:     Uvicorn running on http://0.0.0.0:30549 (Press CTRL+C to quit)


[2025-06-16 12:51:03] INFO:     127.0.0.1:56682 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-16 12:51:03] INFO:     127.0.0.1:56692 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-16 12:51:03] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-16 12:51:04] INFO:     127.0.0.1:56698 - "POST /generate HTTP/1.1" 200 OK
[2025-06-16 12:51:04] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-16 12:51:08] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-16 12:51:08] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.44, #queue-req: 0


[2025-06-16 12:51:08] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 108.54, #queue-req: 0


[2025-06-16 12:51:09] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 105.00, #queue-req: 0
[2025-06-16 12:51:09] INFO:     127.0.0.1:56704 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-06-16 12:51:09] INFO:     127.0.0.1:40328 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-06-16 12:51:09] INFO:     127.0.0.1:40334 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-06-16 12:51:09] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-16 12:51:10] INFO:     127.0.0.1:40348 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-06-16 12:51:10] INFO:     127.0.0.1:40354 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-06-16 12:51:10] Cache flushed successfully!
[2025-06-16 12:51:10] INFO:     127.0.0.1:40358 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-06-16 12:51:10] Start update_weights. Load format=auto
[2025-06-16 12:51:10] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-06-16 12:51:10] Using model weights format ['*.safetensors']


[2025-06-16 12:51:10] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.76it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.76it/s]

[2025-06-16 12:51:11] Update weights end.
[2025-06-16 12:51:11] Cache flushed successfully!
[2025-06-16 12:51:11] INFO:     127.0.0.1:40374 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-06-16 12:51:11] Start update_weights. Load format=auto
[2025-06-16 12:51:11] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-06-16 12:51:11] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-06-16 12:51:11] INFO:     127.0.0.1:40384 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-16 12:51:18] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=38674, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=961228924, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, ba

[2025-06-16 12:51:21] Downcasting torch.float32 to torch.float16.


[2025-06-16 12:51:29] Downcasting torch.float32 to torch.float16.


[2025-06-16 12:51:30] Overlap scheduler is disabled for embedding models.
[2025-06-16 12:51:30] Downcasting torch.float32 to torch.float16.
[2025-06-16 12:51:30] Attention backend not set. Use fa3 backend by default.
[2025-06-16 12:51:30] Init torch distributed begin.


[2025-06-16 12:51:30] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-16 12:51:35] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=27.09 GB


[2025-06-16 12:51:35] INFO:     Started server process [3791724]
[2025-06-16 12:51:35] INFO:     Waiting for application startup.
[2025-06-16 12:51:35] INFO:     Application startup complete.
[2025-06-16 12:51:35] INFO:     Uvicorn running on http://0.0.0.0:38674 (Press CTRL+C to quit)


[2025-06-16 12:51:36] INFO:     127.0.0.1:47602 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-16 12:51:36] INFO:     127.0.0.1:47606 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-16 12:51:36] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-16 12:51:37] INFO:     127.0.0.1:47622 - "POST /encode HTTP/1.1" 200 OK
[2025-06-16 12:51:37] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-06-16 12:51:41] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-16 12:51:41] INFO:     127.0.0.1:58186 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-06-16 12:51:41] Child process unexpectedly failed with exitcode=9. pid=3792230


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-16 12:51:48] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=38900, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1070145771, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downl

[2025-06-16 12:51:59] Overlap scheduler is disabled for embedding models.
[2025-06-16 12:51:59] Attention backend not set. Use flashinfer backend by default.
[2025-06-16 12:51:59] Init torch distributed begin.


[2025-06-16 12:52:00] Init torch distributed ends. mem usage=0.00 GB


[2025-06-16 12:52:01] Load weight begin. avail mem=61.47 GB


[2025-06-16 12:52:02] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.12it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.42it/s]

[2025-06-16 12:52:05] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=46.92 GB, mem usage=14.55 GB.
[2025-06-16 12:52:05] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-06-16 12:52:05] Memory pool end. avail mem=44.13 GB


[2025-06-16 12:52:06] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=29.13 GB


[2025-06-16 12:52:06] INFO:     Started server process [3794737]
[2025-06-16 12:52:06] INFO:     Waiting for application startup.
[2025-06-16 12:52:06] INFO:     Application startup complete.
[2025-06-16 12:52:06] INFO:     Uvicorn running on http://0.0.0.0:38900 (Press CTRL+C to quit)


[2025-06-16 12:52:07] INFO:     127.0.0.1:40474 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-16 12:52:07] INFO:     127.0.0.1:40488 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-16 12:52:07] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-06-16 12:53:12] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-16 12:53:12] INFO:     127.0.0.1:40500 - "POST /encode HTTP/1.1" 200 OK
[2025-06-16 12:53:12] The server is fired up and ready to roll!


[2025-06-16 12:53:26] INFO:     127.0.0.1:55548 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-06-16 12:53:26] Child process unexpectedly failed with exitcode=9. pid=3794967
[2025-06-16 12:53:26] Child process unexpectedly failed with exitcode=9. pid=3794901


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-16 12:53:33] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=30640, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1021711845, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, sleep_on_

[2025-06-16 12:53:44] Attention backend not set. Use flashinfer backend by default.
[2025-06-16 12:53:44] Init torch distributed begin.


[2025-06-16 12:53:44] Init torch distributed ends. mem usage=0.00 GB


[2025-06-16 12:53:45] Load weight begin. avail mem=78.48 GB


[2025-06-16 12:53:45] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:05<00:00,  1.54it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:05<00:00,  1.34it/s]

[2025-06-16 12:53:51] Load weight end. type=Qwen2MoeForCausalLM, dtype=torch.bfloat16, avail mem=51.25 GB, mem usage=27.22 GB.
[2025-06-16 12:53:51] KV Cache is allocated. #tokens: 20480, K size: 1.88 GB, V size: 1.88 GB
[2025-06-16 12:53:51] Memory pool end. avail mem=47.34 GB


[2025-06-16 12:53:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=45.73 GB


[2025-06-16 12:53:52] INFO:     Started server process [3801611]
[2025-06-16 12:53:52] INFO:     Waiting for application startup.
[2025-06-16 12:53:52] INFO:     Application startup complete.
[2025-06-16 12:53:52] INFO:     Uvicorn running on http://0.0.0.0:30640 (Press CTRL+C to quit)
[2025-06-16 12:53:52] INFO:     127.0.0.1:33672 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-16 12:53:53] INFO:     127.0.0.1:33680 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-16 12:53:53] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-16 12:53:54] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1d-gpu-1/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-06-16 12:54:08] Resetting ExpertDistributionRecorder...
[2025-06-16 12:54:08] INFO:     127.0.0.1:33698 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-16 12:54:08] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-16 12:54:09] INFO:     127.0.0.1:33694 - "POST /generate HTTP/1.1" 200 OK
[2025-06-16 12:54:09] The server is fired up and ready to roll!
[2025-06-16 12:54:09] INFO:     127.0.0.1:36864 - "POST /generate HTTP/1.1" 200 OK


[2025-06-16 12:54:09] INFO:     127.0.0.1:46148 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-16 12:54:10] Write expert distribution to /tmp/expert_distribution_recorder_1750078450.9549072.pt
[2025-06-16 12:54:10] Resetting ExpertDistributionRecorder...
[2025-06-16 12:54:10] INFO:     127.0.0.1:46164 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-06-16 12:54:10] Child process unexpectedly failed with exitcode=9. pid=3802048
[2025-06-16 12:54:10] Child process unexpectedly failed with exitcode=9. pid=3801982
